In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def plot(x, y, x_label=None, y_label=None, title=None, name_save_file=None, points=None):
    if title:
        plt.title(title)
    if x_label:
        plt.xlabel(x_label)
    if y_label:
        plt.ylabel(y_label)
    if points:
        for point in points:
            plt.scatter(*point, c="green")
    plt.plot(x, y)
    plt.grid()
    if name_save_file:
        plt.savefig(name_save_file)
    plt.show()

def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

## Next ascent hill climbing в применении к поиску двух витков внутри катушки ##

In [4]:
# размеры витков для первой катушки индуктивности в метрах
coil_1 = np.linspace(0.028, 0.07, 4)

# радиус проволоки
thin = 0.001

start = 0.03 + 2 * thin
finish = 0.09 - 2 * thin
# размеры витков для второй катушки индуктивности в метрах
coil_2 = np.array([0.03, start, finish, 0.09])

# расстояние между катушками в метрах
d = 0.005

In [5]:
def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x


# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

good_mutation = []
bad_mutation = []
all_mutation = []

Q = 2
band = 0.005
thr_k = 1e-4

In [6]:
print("Начальное значение катушки:", coil_2)

i = 0

while True:
    i += 1
    fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
    flag = False
    for q in range(Q):
        coil_2q = coil_2.copy()
        coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        
        # проверка на пересечение витков внутри катушки
        while not test_coils(coil_2q, thin):
            print(f"Есть пересечения внутри катушки! {coil_2q}")
            coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        
        fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
        
        if fit_kq > fit_k:
            coil_2 = coil_2q.copy()
            fit_k = fit_kq
            flag = True
            good_mutation.append((fit_kq, coil_2q))
        else:
            bad_mutation.append((fit_kq, coil_2q))
        all_mutation.append((fit_kq, coil_2q))
    print(f"iter: {i} k = {fit_k}, катушка = {coil_2}")
    if len(good_mutation) >= 2:
        if (good_mutation[-1][0] - good_mutation[-2][0]) / thr_k < 1:
            break

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.4966396342124787, катушка = [0.03       0.03405399 0.08326303 0.09      ]
iter: 2 k = 0.5604440201017764, катушка = [0.03       0.03899505 0.07834656 0.09      ]
iter: 3 k = 0.5880156840896443, катушка = [0.03       0.03899505 0.0750957  0.09      ]
iter: 4 k = 0.5934647035072009, катушка = [0.03       0.03999548 0.0750056  0.09      ]
iter: 5 k = 0.6255468386192595, катушка = [0.03       0.03999548 0.07080381 0.09      ]
iter: 6 k = 0.6294935743544713, катушка = [0.03       0.04097353 0.07080381 0.09      ]
iter: 7 k = 0.6361611858528126, катушка = [0.03       0.04097353 0.06906674 0.09      ]
iter: 8 k = 0.6377306888974933, катушка = [0.03       0.04097353 0.06811312 0.09      ]
iter: 9 k = 0.6377306888974933, катушка = [0.03       0.04097353 0.06811312 0.09      ]
iter: 10 k = 0.6423512054992438, катушка = [0.03       0.04241604 0.06521497 0.09      ]
iter: 11 k = 0.6424903314459244, катушка = [0.03       0.04241604

In [7]:
print(f"kmax = {fit_k}; R2max = {coil_2} м")
print(f"Количество полезных мутаций: {len(good_mutation)}")
print(f"Количество вредных мутаций: {len(bad_mutation)}")
print(f"Всего мутаций: {len(all_mutation)}")

kmax = 0.647174661580134; R2max = [0.03       0.05052734 0.06057413 0.09      ] м
Количество полезных мутаций: 19
Количество вредных мутаций: 11
Всего мутаций: 30
